[Hands on RL Policy Gradient](https://github.com/PacktPublishing/Hands-on-Reinforcement-Learning-with-PyTorch/blob/master/Section%204/4.3%20Policy%20Gradients%20REINFORCE.ipynb)

[Policy Gradient Math](https://towardsdatascience.com/policy-gradients-in-reinforcement-learning-explained-ecec7df94245)

A widely used variation of REINFORCE is to subtract a baseline value from the return to reduce the variance of gradient estimation while keeping the bias unchanged (Remember we always want to do this when possible). For example, a common baseline is to subtract state-value from action-value, and if applied, we would use advantage:

$$
A(s,a) = Q(s,a) - V(s)
$$

in the gradient ascent update. This [post](https://danieltakeshi.github.io/2017/03/28/going-deeper-into-reinforcement-learning-fundamentals-of-policy-gradients/) nicely explained why a baseline works for reducing the variance, in addition to a set of fundamentals of policy gradient.

In [ ]:
#!pip install swig
#!pip install gymnasium[box2d]

## Actor Critic

![Reinforce_bl](acritic.png) 

In [4]:
import torch
from torch import cuda, device, distributions
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#from torch.distributions import Categorical
import math

import gymnasium as gym
import os, random
from pathlib import Path
from collections import deque

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

from IPython.display import clear_output

In [5]:
env_id = "LunarLander-v2"
env = gym.make(env_id)#,render_mode="human")

s_size = env.observation_space.shape[0]
a_size = env.action_space.n

print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample()) # Get a random observation

_____OBSERVATION SPACE_____ 

The State Space is:  8
Sample observation [ 0.8586387  -0.41958687 -0.18232629 -2.5793898  -0.20207646 -3.4008355
  0.39461595  0.91509247]


In [6]:
device = device("cuda:0" if cuda.is_available() else "cpu")

In [7]:
class ActorNet(nn.Module):
    def __init__(self, state_size, action_size, hidden_size):
        super(ActorNet, self).__init__()
        self.dense_layer_1 = nn.Linear(state_size, hidden_size)
        self.dense_layer_2 = nn.Linear(hidden_size, hidden_size)
        self.output = nn.Linear(hidden_size, action_size)
    
    def forward(self, x):
        x = torch.clamp(x,-1.1,1.1)
        x = F.relu(self.dense_layer_1(x))
        x = F.relu(self.dense_layer_2(x))
        return F.softmax(self.output(x),dim=-1) + 1e-8 #-1 to take softmax of last dimension
    
class CriticNet(nn.Module):
    def __init__(self, state_size, hidden_size):
        super(CriticNet, self).__init__()
        self.dense_layer_1 = nn.Linear(state_size, hidden_size)
        self.dense_layer_2 = nn.Linear(hidden_size, hidden_size)
        self.output = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        x = torch.clamp(x,-1.1,1.1)
        x = F.relu(self.dense_layer_1(x))
        x = F.relu(self.dense_layer_2(x))
        return self.output(x)

In [8]:
class ActorCriticAgent():
    def __init__(self, state_size, action_size, hidden_size, actor_lr, critic_lr, discount ):
        self.action_size = action_size
        self.actor_net = ActorNet(state_size, action_size, hidden_size).to(device)
        self.critic_net = CriticNet(state_size, hidden_size).to(device)
        self.actor_optimizer = optim.Adam(self.actor_net.parameters(), lr=actor_lr)
        self.critic_optimizer = optim.Adam(self.critic_net.parameters(), lr=critic_lr)
        self.discount = discount
        
    def select_action(self, state):
        #get action probs then randomly sample from the probabilities
        with torch.no_grad():
            input_state = torch.FloatTensor(state).to(device)
            action_probs = self.actor_net(input_state)
            #detach and turn to numpy to use with np.random.choice()
            action_probs = action_probs.detach().cpu().numpy()
            action = np.random.choice(np.arange(self.action_size), p=action_probs)
        return action

    def train(self, state_list, action_list, reward_list, next_state_list, done_list):
        
        # create tensors
        state_t = torch.FloatTensor(state_list).to(device)
        next_state_t = torch.FloatTensor(next_state_list).to(device)
        action_t = torch.LongTensor(action_list).to(device).view(-1,1)
        reward_t = torch.FloatTensor(reward_list).to(device).view(-1,1)
        done_t = torch.FloatTensor(done_list).to(device).view(-1,1) #flipping 0 and 1 in RL training loop
        
        # get critic estimate
        critic_t = self.critic_net(state_t).view(-1,1)
        with torch.no_grad():
            critic_td_t = reward_t + done_t * self.discount * self.critic_net(next_state_t).view(-1,1)
            advantage_t = critic_td_t - critic_t
        
        # calculate actor loss
        selected_action_prob = self.actor_net(state_t).gather(1, action_t)
        actor_loss = torch.mean(-torch.log(selected_action_prob) * advantage_t)
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step() 

        # calculate critic loss
#         loss_fn = nn.MSELoss()
#         critic_loss = loss_fn(critic_t, critic_td_t)
        critic_loss = F.smooth_l1_loss(critic_t, critic_td_t)
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step() 
        return actor_loss.detach().cpu().numpy(), critic_loss.detach().cpu().numpy()

In [13]:
hidden_layer = 64
gamma = 0.995
actor_lr = 0.001
critic_lr = 0.001
episodes = 100_000
avg_win_size = 50
epi_results = deque(maxlen=avg_win_size)

agent = ActorCriticAgent(s_size, a_size, hidden_layer, actor_lr, critic_lr,  gamma)

In [31]:
stats_rewards_list = [] # store stats for plotting in this
stats_every = 10 # print stats every this many episodes
total_reward = 0

episode_length = 0
stats_actor_loss, stats_critic_loss = [], []

for epi in range(100_000):
    done = False
    state = env.reset()[0]
    state_list, action_list, reward_list, next_state_list, done_list = [], [], [], [], []

    time_steps = 0

    # train in each episode until episode is done
    while not done:
       
        action = agent.select_action(state)
        
        # enter action into the env
        next_state, reward, done, _, _ = env.step(action)
        total_reward += reward
        episode_length += 1
        # end episode early
        if total_reward < -250:
            done = 1
        # store agent's trajectory
        state_list.append(state)
        action_list.append(action)
        reward_list.append(reward)
        next_state_list.append(next_state)
        done_list.append(1. - float(done))
        
        # train the agent
        
        state = next_state
        time_steps+=1
        
    epi_results.append(np.sum(reward_list))
    actor_loss, critic_loss = agent.train(state_list, action_list, reward_list, next_state_list, done_list)
    total_reward = 0

    if epi%100==0:
        clear_output()
    if epi%10==0:
        print(f'epi:{epi:05d} reward:{np.sum(reward_list):8.2f} timesteps:{time_steps}')
    if np.mean(np.mean(epi_results))>200:
        break

epi:42700 reward:   38.14 timesteps:225
epi:42710 reward:  275.75 timesteps:336
